#**Space Weather and Satellite Risk Analytics**

#**1. Importing required Libraries**

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#**2. Creating the required CSV dataset directly and loading in Pandas**

In [2]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(42)

#1. Solar Flares Dataset (10 rows)

dates = pd.date_range(start="2023-01-01", periods=10, freq='D')
flare_classes = ['C', 'M', 'X']
flare_probs = [0.7, 0.25, 0.05]
flare_intensity = {'C': (1, 10), 'M': (10, 50), 'X': (50, 100)}

solar_flares = pd.DataFrame({
    'Date': dates,
    'Flare_Class': np.random.choice(flare_classes, len(dates), p=flare_probs)
})
solar_flares['Flare_Intensity'] = solar_flares['Flare_Class'].apply(lambda x: np.round(np.random.uniform(*flare_intensity[x]),2))
solar_flares.to_csv('solar_flares_sample.csv', index=False)


#2. Geomagnetic Indices Dataset (10 rows)

geomagnetic_index = pd.DataFrame({
    'Date': dates,
    'Kp_Index': np.random.randint(0, 10, len(dates)),
    'Ap_Index': np.random.randint(0, 400, len(dates))
})
geomagnetic_index.to_csv('geomagnetic_indices_sample.csv', index=False)


#3. Satellite Anomalies Dataset (10 rows)

satellites = ['Sat-A', 'Sat-B', 'Sat-C', 'Sat-D', 'Sat-E']
anomaly_types = ['Power', 'Communication', 'Sensor', 'Orbit']
anomaly_dates = np.random.choice(dates, 10)

satellite_anomalies = pd.DataFrame({
    'Date': anomaly_dates,
    'Satellite': np.random.choice(satellites, 10),
    'Anomaly_Type': np.random.choice(anomaly_types, 10),
    'Severity': np.random.randint(1, 5, 10)
})
satellite_anomalies.to_csv('satellite_anomalies_sample.csv', index=False)

# Previewing the  datasets
solar_flares, geomagnetic_index, satellite_anomalies

(        Date Flare_Class  Flare_Intensity
 0 2023-01-01           C             1.19
 1 2023-01-02           X            98.50
 2 2023-01-03           M            43.30
 3 2023-01-04           C             2.91
 4 2023-01-05           C             2.64
 5 2023-01-06           C             2.65
 6 2023-01-07           C             3.74
 7 2023-01-08           M            30.99
 8 2023-01-09           C             4.89
 9 2023-01-10           M            21.65,
         Date  Kp_Index  Ap_Index
 0 2023-01-01         9       328
 1 2023-01-02         2       166
 2 2023-01-03         6       273
 3 2023-01-04         3       387
 4 2023-01-05         8        88
 5 2023-01-06         2       315
 6 2023-01-07         4        13
 7 2023-01-08         2       241
 8 2023-01-09         6       264
 9 2023-01-10         4       345,
         Date Satellite   Anomaly_Type  Severity
 0 2023-01-05     Sat-D  Communication         1
 1 2023-01-02     Sat-B          Orbit         2
 2 2

#**3. Merging and Preprocessing Datasets**

In [3]:
# Merging satellite anomalies with solar flares on Date
merged_df = satellite_anomalies.merge(solar_flares, on='Date', how='left')

# Merging the above with geomagnetic indices
merged_df = merged_df.merge(geomagnetic_index, on='Date', how='left')

# Fill any potential NaN values with 0
merged_df = merged_df.fillna(0)

# Preview merged dataset
merged_df.head()

,Date,Satellite,Anomaly_Type,Severity,Flare_Class,Flare_Intensity,Kp_Index,Ap_Index
0,2023-01-05,Sat-D,Communication,1,C,2.64,8,88
1,2023-01-02,Sat-B,Orbit,2,X,98.50,2,166
2,2023-01-04,Sat-B,Communication,4,C,2.91,3,387
3,2023-01-07,Sat-D,Sensor,1,C,3.74,4,13
4,2023-01-08,Sat-E,Orbit,4,M,30.99,2,241


#**4. Calculating Risk Score**

In [4]:
# Defining the weights
alpha = 0.6  # weight for flare intensity
beta = 0.4   # weight for Kp index

# Calculating Risk Score
merged_df['Risk_Score'] = alpha * merged_df['Flare_Intensity'] + beta * merged_df['Kp_Index'] * 10

# Preview
merged_df[['Date','Satellite','Anomaly_Type','Flare_Class','Flare_Intensity','Kp_Index','Risk_Score']]

,Date,Satellite,Anomaly_Type,Flare_Class,Flare_Intensity,Kp_Index,Risk_Score
0,2023-01-05,Sat-D,Communication,C,2.64,8,33.584
1,2023-01-02,Sat-B,Orbit,X,98.50,2,67.100
2,2023-01-04,Sat-B,Communication,C,2.91,3,13.746
3,2023-01-07,Sat-D,Sensor,C,3.74,4,18.244
4,2023-01-08,Sat-E,Orbit,M,30.99,2,26.594
5,2023-01-03,Sat-B,Sensor,M,43.30,6,49.980
6,2023-01-01,Sat-B,Orbit,C,1.19,9,36.714
7,2023-01-04,Sat-D,Communication,C,2.91,3,13.746
8,2023-01-02,Sat-B,Sensor,X,98.50,2,67.100
9,2023-01-08,Sat-B,Orbit,M,30.99,2,26.594


#**5. Structuring the KPI  s**

In [5]:
# KPI 1: Average Risk Score per Satellite
satellite_risk = merged_df.groupby('Satellite')['Risk_Score'].mean().reset_index().sort_values(by='Risk_Score', ascending=False)

# KPI 2: Count of Anomalies per Flare Class
flare_anomalies = merged_df.groupby('Flare_Class')['Anomaly_Type'].count().reset_index()
flare_anomalies.columns = ['Flare_Class', 'Anomaly_Count']

satellite_risk, flare_anomalies

(  Satellite  Risk_Score
 0     Sat-B      43.539
 2     Sat-E      26.594
 1     Sat-D      21.858,
   Flare_Class  Anomaly_Count
 0           C              5
 1           M              3
 2           X              2)

#**6. Data Visualisation**

#A. Visualizing Risk per Satellite

In [6]:
import plotly.express as px

# Bar chart: Average Risk Score per Satellite
fig1 = px.bar(satellite_risk, x='Satellite', y='Risk_Score',
              color='Risk_Score', color_continuous_scale='Blues',
              title='Average Satellite Risk Score')
fig1.show()

#B. Flare Class vs Number of Anomalies

In [7]:
# Pie chart: Number of Anomalies by Flare Class
fig2 = px.pie(flare_anomalies, values='Anomaly_Count', names='Flare_Class',
              title='Anomalies Distribution by Solar Flare Class',
              color_discrete_sequence=px.colors.sequential.RdBu)
fig2.show()

#C. Scatter Plot - Flare Intensity vs Risk Score

In [8]:
# Scatter plot: Flare Intensity vs Risk Score
fig3 = px.scatter(merged_df, x='Flare_Intensity', y='Risk_Score',
                  color='Satellite', size='Severity',
                  hover_data=['Anomaly_Type', 'Kp_Index'],
                  title='Flare Intensity vs Satellite Risk Score')
fig3.show()

#D. Heatmap - Kp Index vs Satellite

In [9]:
# Create pivot table for heatmap
heatmap_df = merged_df.pivot_table(values='Risk_Score', index='Satellite', columns='Date', fill_value=0)

fig4 = px.imshow(heatmap_df,
                 color_continuous_scale='Blues',
                 title='Satellite Risk Score Heatmap Over Time')
fig4.show()

#**7. Dashboard**

In [10]:
import plotly.io as pio
pio.renderers.default = "colab"

In [11]:
!pip install --upgrade plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 70.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, HBox
from IPython.display import display
import plotly.io as pio

# Running plotly
pio.renderers.default = "colab"

# Loading data
solar_flares = pd.read_csv('solar_flares_sample.csv')
geomagnetic_index = pd.read_csv('geomagnetic_indices_sample.csv')
satellite_anomalies = pd.read_csv('satellite_anomalies_sample.csv')

# Merging datasets
merged_df = satellite_anomalies.merge(solar_flares, on='Date', how='left')
merged_df = merged_df.merge(geomagnetic_index, on='Date', how='left')
merged_df = merged_df.fillna(0)
merged_df["Date"] = pd.to_datetime(merged_df["Date"], errors="coerce")

# Risk Score
alpha, beta = 0.6, 0.4
merged_df['Risk_Score'] = alpha * merged_df['Flare_Intensity'] + beta * merged_df['Kp_Index'] * 10

# Extra time breakdowns
merged_df['month'] = merged_df['Date'].dt.to_period("M")
merged_df['week'] = merged_df['Date'].dt.to_period("W")
merged_df['weekday'] = merged_df['Date'].dt.day_name()

# Dropdown options
satellite_options = ['All'] + merged_df['Satellite'].unique().tolist()
flare_options = ['All'] + merged_df['Flare_Class'].unique().tolist()

# Global theme state
theme_state = {"template": "plotly_white"}

# Filter function
def filter_data(selected_satellite, selected_flare):
    df = merged_df.copy()
    if selected_satellite != 'All':
        df = df[df['Satellite'] == selected_satellite]
    if selected_flare != 'All':
        df = df[df['Flare_Class'] == selected_flare]
    return df

# Dashboard update
def update_dashboard(selected_satellite, selected_flare):
    df = filter_data(selected_satellite, selected_flare)
    if df.empty:
        print(f"No data for Satellite: {selected_satellite} & Flare Class: {selected_flare}")
        return

    # --- KPIs ---
    max_risk = df['Risk_Score'].max()
    busiest_date = df.loc[df['Risk_Score'].idxmax(), 'Date'].strftime("%d-%m-%Y")
    popular_flare = df['Flare_Class'].mode()[0] if not df['Flare_Class'].empty else "N/A"
    flare_counts = df['Flare_Class'].value_counts()

    # --- Aggregations ---
    daily = df.groupby('Date')['Risk_Score'].mean().reset_index()
    monthly = df.groupby('month')['Risk_Score'].mean().reset_index()
    monthly['month'] = monthly['month'].astype(str)
    weekly = df.groupby('week')['Risk_Score'].mean().reset_index()
    weekly['week'] = weekly['week'].astype(str)
    weekday_avg = df.groupby('weekday')['Risk_Score'].mean().reindex(
        ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])
    flare_dist = flare_counts.reset_index()
    flare_dist.columns = ['Flare_Class','Count']
    heatmap_df = df.groupby(['Date','Satellite'])['Risk_Score'].mean().reset_index()

    # --- Dashboard Layout ---
    dashboard = make_subplots(
        rows=3, cols=3,
        subplot_titles=("Daily Avg Risk","Monthly Avg Risk","Weekly Avg Risk",
                        "Avg Risk by Weekday","Flare Class Distribution","Satellite Risk Heatmap",
                        "Max Risk Score","Highest Risk Date","Most Frequent Flare Class"),
        specs=[[{"type":"xy"},{"type":"xy"},{"type":"xy"}],
               [{"type":"bar"},{"type":"domain"},{"type":"heatmap"}],
               [{"type":"indicator"},{"type":"indicator"},{"type":"indicator"}]],
        horizontal_spacing=0.08,
        vertical_spacing=0.12
    )

    # Row 1
    for trace in px.line(daily, x='Date', y='Risk_Score').data:
        dashboard.add_trace(trace, row=1, col=1)

    # Monthly as Bubble Chart
    dashboard.add_trace(go.Scatter(
        x=monthly['month'],
        y=monthly['Risk_Score'],
        mode='markers',
        marker=dict(
            size=monthly['Risk_Score'],       # bubble size proportional to risk
            sizemode='area',
            sizeref=2.*max(monthly['Risk_Score'])/(40.**2),  # auto-scale
            color=monthly['Risk_Score'],
            colorscale='Plasma',
            showscale=True
        ),
        text=[f"Month: {m}<br>Risk: {r:.2f}" for m,r in zip(monthly['month'], monthly['Risk_Score'])],
        name="Monthly Avg Risk"
    ), row=1, col=2)

    for trace in px.line(weekly, x='week', y='Risk_Score').data:
        dashboard.add_trace(trace, row=1, col=3)

    # Row 2
    dashboard.add_trace(go.Bar(x=weekday_avg.index, y=weekday_avg.values), row=2, col=1)

    # Pie chart with side labels
    dashboard.add_trace(go.Pie(
        labels=flare_dist['Flare_Class'],
        values=flare_dist['Count'],
        textinfo='label+percent+value',
        textposition='outside',
        automargin=True,
        hole=0.3,
        marker=dict(line=dict(color='white', width=2))
    ), row=2, col=2)

    dashboard.add_trace(go.Heatmap(
        z=heatmap_df['Risk_Score'],
        x=heatmap_df['Date'],
        y=heatmap_df['Satellite'],
        colorscale='Viridis'
    ), row=2, col=3)

    # Row 3 - KPIs
    dashboard.add_trace(go.Indicator(mode="number", value=max_risk,
                                     title={"text":"Max Risk Score"}), row=3, col=1)
    dashboard.add_trace(go.Indicator(mode="number", value=pd.to_datetime(busiest_date).day,
                                     title={"text":f"Highest Risk Date ({busiest_date})"}), row=3, col=2)
    dashboard.add_trace(go.Indicator(mode="number", value=flare_counts[popular_flare],
                                     title={"text":f"Most Frequent Flare: {popular_flare}"}), row=3, col=3)

    # Layout
    dashboard.update_layout(
        template=theme_state["template"],
        title="<b><span style='font-size:28px'>Space Weather & Satellite Risk Dashboard</span></b>",
        title_x=0.5,
        showlegend=True,
        height=1000, width=1500
    )

    dashboard.show()

# Dropdowns & Buttons
dropdown_satellite = widgets.Dropdown(options=satellite_options, description='Satellite:', value='All')
dropdown_flare = widgets.Dropdown(options=flare_options, description='Flare Class:', value='All')
btn_toggle_theme = widgets.Button(description="Toggle Theme", button_style='info')

def toggle_theme(b):
    theme_state["template"] = "plotly_dark" if theme_state["template"]=="plotly_white" else "plotly_white"
    update_dashboard(dropdown_satellite.value, dropdown_flare.value)

btn_toggle_theme.on_click(toggle_theme)

controls = HBox([dropdown_satellite, dropdown_flare, btn_toggle_theme])
display(controls)

# Initial render
update_dashboard('All','All')

$$..........................Thank You..........................$$